In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cap-4611-2021-fall-assignment-3/sample_submission.csv
/kaggle/input/cap-4611-2021-fall-assignment-3/eval.csv
/kaggle/input/cap-4611-2021-fall-assignment-3/train.csv


In [2]:
train_df = pd.read_csv("/kaggle/input/cap-4611-2021-fall-assignment-3/train.csv")
test_df = pd.read_csv("/kaggle/input/cap-4611-2021-fall-assignment-3/eval.csv")

In [3]:
train_df.head()

,id,0,1,2,3,4,5,6,7,8,...,1267,1268,1269,1270,1271,1272,1273,1274,pubchem_id,Eat
0,0,73.516695,21.340907,18.686189,16.515660,13.986340,13.724160,13.546577,13.514781,13.483112,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67498,-6.814462
1,1,388.023441,27.882166,20.729171,20.649087,20.449092,18.762317,18.311063,16.011055,15.832926,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65270,-8.776741
2,2,73.516695,20.511719,20.511717,18.642329,18.640126,18.632230,18.630913,14.895390,14.894787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3022,-13.543936
3,3,53.358707,16.499291,16.424568,15.764738,15.324091,15.321921,13.798724,13.726431,13.707620,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,66725,-12.847512
4,4,53.358707,15.265913,15.191498,15.190635,13.687368,13.686528,13.653668,13.653627,13.653229,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62436,-17.290178


In [4]:
test_df.head()

,id,0,1,2,3,4,5,6,7,8,...,1266,1267,1268,1269,1270,1271,1272,1273,1274,pubchem_id
0,0,73.516695,20.742464,20.661153,18.722844,14.893199,13.840885,13.840738,13.653650,13.653083,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72871
1,1,36.858105,12.890223,12.827174,12.812343,12.810806,12.690070,12.526905,12.525781,12.525523,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39066
2,2,73.516695,23.608635,23.607562,20.510765,20.507700,18.560693,18.559518,17.736690,17.736332,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43704
3,3,73.516695,20.631428,20.630475,20.606247,17.871111,17.863434,16.019955,16.019584,15.988977,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48781
4,4,73.516695,20.766917,14.263870,12.965133,12.787015,12.688717,12.560287,12.459889,12.458895,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32240


In [5]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.model_selection import RandomizedSearchCV, KFold

In [6]:

from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

ids = test_df['id']

X = pd.get_dummies(train_df.drop(columns=["Eat","id","pubchem_id"]))

y = train_df['Eat']


print(X)
print(y)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = .25, random_state = 42)

from sklearn.preprocessing import StandardScaler
from keras import backend as K

#scaler = StandardScaler().fit(X_train)

#X_train = scaler.transform(X_train)

#X_test = scaler.transform(X_test)

model = Sequential()

model.add(Dense(200, activation='sigmoid', input_shape=(1275,)))
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(50, activation='sigmoid'))
model.add(Dense(1))

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred-y_true)))

model.compile(loss=root_mean_squared_error, optimizer='adam', metrics=['mse'])

model.fit(X_train, y_train, epochs=100, batch_size=20, verbose = 1)
#validation_data = (X_test,y_test)

X_test = pd.get_dummies(test_df.drop(columns=["id","pubchem_id"]))

print(X_test)
y_pred = model.predict(X_test)
score = model.evaluate(X_test, y_test, verbose = 1)
print(score)

model.summary()

y_pred = y_pred.reshape(len(y_pred))
output = pd.DataFrame({'id': ids, 'Eat': y_pred})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")
print(output.to_string())

                0          1          2          3          4          5  \
0       73.516695  21.340907  18.686189  16.515660  13.986340  13.724160   
1      388.023441  27.882166  20.729171  20.649087  20.449092  18.762317   
2       73.516695  20.511719  20.511717  18.642329  18.640126  18.632230   
3       53.358707  16.499291  16.424568  15.764738  15.324091  15.321921   
4       53.358707  15.265913  15.191498  15.190635  13.687368  13.686528   
...           ...        ...        ...        ...        ...        ...   
12988   73.516695  20.703867  14.234935  12.691563  12.610370  12.467563   
12989   53.358707  16.499601  16.498606  16.468071  16.466495  15.736080   
12990   73.516695  21.106224  20.673147  16.147199  16.141857  14.168558   
12991   73.516695  20.780679  18.702405  18.560051  17.883523  17.779027   
12992   36.858105  12.495330  12.495178  12.495178  12.494212  12.185325   

               6          7          8          9  ...  1265  1266  1267  \
0      13.5

2022-02-11 21:13:37.095932: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-11 21:13:37.190071: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-11 21:13:37.190736: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-11 21:13:37.193756: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/100
488/488 [==============================] - 2s 3ms/step - loss: 4.0586 - mse: 21.3896
Epoch 2/100
488/488 [==============================] - 1s 2ms/step - loss: 0.6460 - mse: 0.5425
Epoch 3/100
488/488 [==============================] - 1s 2ms/step - loss: 0.3816 - mse: 0.1694
Epoch 4/100
488/488 [==============================] - 1s 2ms/step - loss: 0.3273 - mse: 0.1234
Epoch 5/100
488/488 [==============================] - 1s 2ms/step - loss: 0.3079 - mse: 0.1105
Epoch 6/100
488/488 [==============================] - 1s 2ms/step - loss: 0.2868 - mse: 0.0956
Epoch 7/100
488/488 [==============================] - 1s 2ms/step - loss: 0.2887 - mse: 0.0967
Epoch 8/100
488/488 [==============================] - 1s 2ms/step - loss: 0.2832 - mse: 0.0929
Epoch 9/100
488/488 [==============================] - 1s 2ms/step - loss: 0.2810 - mse: 0.0924
Epoch 10/100
488/488 [==============================] - 1s 2ms/step - loss: 0.2728 - mse: 0.0867
Epoch 11/100
488/488 [================